# Spectre Ops
> Triage for v10 of this commodity implant
- toc: true 
- badges: true
- categories: [spectreops,config,strings,cpp]

## Overview

Spectre RAT is commodity malware designed for targetted (e-crime) attacks. According to an advert for the malware the RAT capabilites closely mimic those of a commercial implants such as cobalt strike, with the ability to harvest data from victims, and deploy second stage payloads (ref. [Wallmart Global Tech Blog](https://medium.com/walmartglobaltech/spectre-spc-v9-campaigns-and-updates-546c2e65e247))

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/0*-kX-TgUS74Rp8R_D)

### References
- [Spectre v4.0 - YOROI](https://yoroi.company/research/spectre-v4-0-the-speed-of-malware-threats-after-the-pandemics/)
- [Malpedia](https://malpedia.caad.fkie.fraunhofer.de/details/win.spectre)
- [Spectre (SPC) v9 Campaigns and Updates](https://medium.com/walmartglobaltech/spectre-spc-v9-campaigns-and-updates-546c2e65e247)

### Sample
- packed `74f30466c1edb8daf729df5531e6047e69cfbcd0625fbb3df02bb655ec90c480` [UnpacMe](https://www.unpac.me/results/1e3418c6-b4c7-4a7f-90ab-887168426407)
- unpacked `734a22c413872a8b5f6fc5342261b9ff2c256c1891bb258747ff000df960ce4f` [UnpacMe](https://www.unpac.me/results/1e3418c6-b4c7-4a7f-90ab-887168426407)

## Analysis
We can dump the encrypted strings using x64db and then import them into IDA.

In [ ]:
strings = {0x580A7C: "76E894005c2DE86E40b032a0931D2ABC05C6eB36ACb1C18F5b640aD24Bbc9454",
0x19FEC8: "OzYuOT02LjY1LDUw",
0x19FEE0: "ZWN0bXtjYXJtZ2xjaXxjbWFya28sYW9t",
0x19FEF8: "Y2xnbWRpbmFpaGRmZnpnZHJpYWssYW9t",
0x58098C: "1950BC4F01",
0x5806F8: "17B4C29833",
0x58080C: "EEE592271B",
0x580590: "CullinetProgram",
0x580B90: "680FDC",
0x580578: "ACDB39",
0x580A34: "09-23",
0x580860: "rhnu.dll",
0x580650: "nyxhv",
0x5805D8: "B3C830CA-4433-CC3A-6737",
0x5809A4: "uhapy",
0x5808F0: "http://manjitaugustuswaters.com",
0x580740: "jnml.php",
0x580638: "grfq.php",
0x580698: "tsml.zip",
0x580A4C: "tsml_nonir.zip",
0x580BF0: "wvxk.zip",
0x580B0C: "wvxk_x64.zip",
0x580B78: "wsau.exe",
0x5805C0: "nico=",
0x580B3C: "&yfat=",
0x580A04: "&zbce=",
0x580AAC: "&qiob=",
0x5808A8: "&jwrb=",
0x5807AC: "&nsmb=",
0x5806B0: "&inau=",
0x580608: "&wpof=",
0x58077C: "&chja=",
0x5809BC: "&ehin=",
0x5808C0: "&vmzn=",
0x5809EC: "&ouej=",
0x580944: "&rzya=",
0x580890: "&cdyt=",
0x58092C: "&rich=",
0x580794: "&clsx=",
0x580ADC: "&hwqy=",
0x5805A8: "?selk=",
0x580BD8: "vdle",
0x580BC0: "down/",
0x580560: "\\Microsoft\\Windows\\Start Menu\\Programs\\Startup\\",
0x58083C: "nircmdc.exe",
0x580BA8: "zip.exe",
0x580680: "/c ping localhost -n 6 > nul &",
0x580974: "/c ping localhost -n 10 > nul &",
0x5805F0: "cout",
0x5807F4: "http://",
0x580AC4: "true",
0x580908: "false",
0x5809D4: "void",
0x580A94: ".asd",
0x580620: "[@]",
0x5808D8: "[|]",
0x5807DC: "[*]",
0x580710: ".png",
0x580668: ".exe",
0x580B54: ".lnk",
0x580764: ".vbs",
0x580B24: ".txt",
0x580728: ".7z",
0x5806E0: ".bak",
0x580A1C: " --headless=old --disable-gpu --remote-debugging-port=0 ",
0x5807C4: "MyTasks\\"}

import idaapi
import idautils
import ida_bytes
import idc
import ida_kernwin
import json
import string
import ida_loader


def set_hexrays_comment(address, text):
    '''
    set comment in decompiled code
    '''
    try:
        cfunc = idaapi.decompile(address)
        tl = idaapi.treeloc_t()
        tl.ea = address
        tl.itp = idaapi.ITP_SEMI
        if cfunc is not None:
            cfunc.set_user_cmt(tl, text)
            cfunc.save_user_cmts() 
    except:
        print(f"Unable to comment pseudocode at {hex(address)}")


def set_comment(address, text):
    ## Set in dissassembly
    idc.set_cmt(address, text,0)
    ## Set in decompiled data
    set_hexrays_comment(address, text)


for k in strings.keys():
    s = strings[k]
    print(f"{hex(k)}: {s}")
    ida_name.set_name(k, 'g_str_' + s,  ida_name.SN_FORCE)
    set_comment(k, s)



### C2 Communication

The decrypted string `76E894005c2DE86E40b032a0931D2ABC05C6eB36ACb1C18F5b640aD24Bbc9454` is used to encrypt/decrypt C2 traffic, as well as the hard coded C2 hosts.

In [6]:
import base64
import urllib.parse

def decrypt(data, key):
    data = urllib.parse.unquote(data)
    data = base64.b64decode(data)

    out = []
    for i in range(len(data)):
        out.append(data[i] ^ key[i % len(key)] & 0xa)
    return bytes(out)

data_list = [b'ZWN0bXtjYXJtZ2xjaXxjbWFya28sYW9t',
        b'Y2xnbWRpbmFpaGRmZnpnZHJpYWssYW9t',
        b'OzYuOT02LjY1LDUw',
        b'MzIwIjkwMjE0KEE6XF1xZXJzXkJwd25vVENwcEZhdmNcUm1jbWtsZVxBd2xuaWZndFJwb2dyYW9cQDFBMDMwQ0MvNDw7My1DQzFDLTY/MTdbQF8xMjIqMTgyMTEoOTYsMTU0LDY3LDUwWUJdMzA4KDEyMjkzKndmL212am1yW0BfMzA4IjEwMDEzKDk0JjM1Ni40NSw1MFtIXzIqMDA3KEM6XldzZ3BxXEBwdWxvVENwckZhdGFcUG9jb2tmZ1xDd25sYWZldFBybWVyYWVed3Nhdy5nemVbSF8xMDIqMzIwMTMoOTYsMzU0LDY3Lj8yW0JfMioyMDcqQTheXXNlcnFeQnp9bm9cQXJyRGF8Y1xSb2Nta2xnXEt3bGxrbmd2UHJtZXJjb150cW9sLHphcltCXzEwMCozMDMwMSJDOlxXcWV6e1xCcnVsbVxBeHJEYXRjXFBtYW1hbGdcQXVubmluZ3ZQcG1lcmNvXHZzZW4ueGtwW0BdMzAyKDM4MDExKDs0Jjk1Ni42Nyw3MFNCXTEwMiozMjExPihbQF8=',
        b'WUJdU0hdW0BdWUJddHp3ZVtAX2ZjbnNlU0JddjMwXXF0bHBdeDo0XXBjbGFvX3A6Nl0wMDI0LTI5LzA2VzE4LTE6Lm1wZSoxW0JfMTAiWUBd',
        b'dDMwV3t0bHJfejo2X3hjbmFtXXg6NF8yODA0LTI5LzA0XzM6LTE6LGV6Zyp0MThdc3Zucl94ODRfcmNsaW1feDo0Xzo4MjQtMDsvMjRXMzgtMzouZ3plKj4oMSooNzExMTkyM2RmYzI5OjMyYWI8YzBgNDIzODhnYmZjZmsqMA==',
        b'MzIwIjkwMDExKDs0Ljk3Ni42Ny41MltAVTAqMjI1KEE6XFdxZXBxXkJwd25tXElycEZjdGFcUm1hb2tsb1xDdW5uaWZtdFByb2VwYW1UdXZ4ayx6a3JbQFUzMDAoMTIzMjMoQTpeV3FlcHFcQHJ9bG9eQ3BwRGF2YV5QbWltaW5lXkN9ZGxpbmV2UnJvb3BhbVx1dnppLnphcltAXw%3D%3D',
        b'MzA4IjkwMipDOF5Vc21wc1xCcHVsbVxBeHJEYXZhXlBvYW9rbmVeQXVubmlsZXxScm1lcmFtXDJFNEY2PzIyNkc3RDpMQjY4RkBGRDhNNzU1M0QzRl1GSURHUy41ellCXTEyMiozMjIxMyg5Ni45NzYsNDUuNzBZQF8zMD8qODE6KEMyVFVzZXJxXkJyfWxvXEFycEZjdGFUTm9jY2xeRW9vZW5lXkFqcm1vZV5Ve2dyIkZhdGFcUnJtZGtkZSAxWUJdOTo3KjgxOihDOlRXc2VycVxAcHVuZ15BcHJEY3ZhXE5tY2NuXkdtbWduZVRBaHBtbWVcVXFlcCJGaXRhXFJwb25hbGUgMllCXTE6NSo4MToqQThcVXtncnNeQnB3bm9eQ3ByRmN0Y15MbWNpblxFbW9nbGVeQ2pwbWVlXFVxZ3IoTGF0YVxScG9mYW5lIDNZQF8zMjciOjE4KEM4XlVzZ3BzXkBwdWxtXENweEZhdmNcTG9jY2xeRW1nZ2xlXkFoemdtZVxVcWdyIExjdGFcUnJtZGlsbSI0W0JdMzA3KjozOChBOFxXcWVwc1RAcndsb1xBcHJEY3ZjVExvY2NuXE9nb2dsZV5BaHJnb2VcVXFlcCJEYXxjXFBwb2RrbGUiN1tCXzMwMigxMjA5Myo7Ni4xNTYsNjcsNThbQF0zMDciMDE4KkM4XlVzbXBzXEJwdWxtXEF4ckRhdmFeTm9jY25cT2thcm1xb2R0VEdkZWdcVXNlcCBGY3ZpXFBybWRpZG0gMVtAXzMyNyI6MTgqQTpeV3NlenFcQnB1bG1cQXJyRGN2Y1xObWNjbFRPaWFwb3NvZnZcR2ZlbVxVc2dwIExpdGFcUHBtZmlkZyAyW0JdMzA3KjAzOCpBOl5Xc2VwcVxAcHdubV5BcnBMY3RjXkxvY2FuXE9rYXpvc29kdlxNbGdlXFVxZ3IgTGN0YVxScm1kaWxtIjNbQl0zMDcqOjM4KEE4XFdxZXBzVEByd2xvXEFwckRjdmNUTG9jY25cRWFjcm9zbWR0XE1mZ2VcV3NncCBEaXZhXFJybWRpbGciNFlCXzEwNSo6MTAoQzheVXNlcnFcQHB3Zm9cQXJyRGl8YVxMb2FjbFxFa2Nyb3FvZHZcRWxlZVxXc2dwIERjdmFeUnBvZGtsZyA9WUBfMzI3KjgzOChBOFRVc2VwcVxKenVub1xDcnBEaXZhXFJtYW9rbmdUTXBlcGEiUW9mdnVhcGdeT3JncmMgW3ZhYG5lXERlZGF3bnZTQF0xMDUqMDk4KkM6XldzZXpxXEJyd25tXkFweEZhdGNcUG1hbWtsZ15NcmVwYyBRb252d2NwZVxPcGdyYyJRfGFibGdeUHpnZmlsZSIzW0BVMzI3KjoxOihDOlRXc2Vwc15AcnVsbVxDcnJEY3ZhXlJnY21rbGdcT3BncmMiUWdmdHdjcGVUR3BlcmEiUXRham5lXFBwb2RrbGUoMFtAXzEwNSo4MzoqQTheVXFncnFcSnB1bG1cQXBwRmF2Y15ab2Fta2xnVEdwZXJhIlFvZnx1YXJlXk9yZ3JhKFF0YWBsZ15Qcm1kaW5nIjNZQl0zMj8oODM6KkM6XFdzZ3BxVEJydWxtXEl4cERhdGNeUm9pb2luZ15PcmdyYShRb2Z2d2NwZVxNcmVwYyJTdmNibmVUUnJtZGlsZSA2W0JfMzo3KjgzOipLMlxVc2VwcVxCendub1xDcHJGYXRpXlJvY21rbGdcTXJlcGMiU21kdHVhemdcTXJlcmEgUXRjYG5tXFBybWRpZG0gNVtAXzMyNyI6MjgqWUBf']
        
key = b'76E894005c2DE86E40b032a0931D2ABC05C6eB36ACb1C18F5b640aD24Bbc9454'


for data in data_list:
    print(decrypt(data, key))

b'gatescarmencitamario.com'
b'angelinaijfffredrick.com'
b'94.156.65.70'
b'100*10214*C:\\Users\\Bruno\\AppData\\Roaming\\CullinetProgram\\B3C830CA-4433-CC3A-6737[@]100*10011*94.156.65.70[@]100*10093*wd/other[@]100*10011*94.156.65.70[@]2*205*C:\\Users\\Bruno\\AppData\\Roaming\\CullinetProgram\\wsau.exe[@]100*10011*94.156.65.70[@]2*205*C:\\Users\\Bruno\\AppData\\Roaming\\CullinetProgram\\tsml.zip[@]100*10123*C:\\Users\\Bruno\\AppData\\Roaming\\CullinetProgram\\tsml.zip[@]100*10011*94.156.65.70[@]100*10116*[@]'
b'[@][@][@][@]true[@]false[@]v10_stlr_x86_panam_x86_2024-09-24_18-38.exe*1[@]10*[@]'
b'v10_stlr_x86_panam_x86_2024-09-24_18-38.exe*v10_stlr_x86_panam_x86_2024-09-24_18-38.exe*6*1**7331901dda09812cb4a0b62388ebdadc*0'
b'100*10011*94.156.65.70[@]2*205*C:\\Users\\Bruno\\AppData\\Roaming\\CullinetProgram\\wvxk.zip[@]100*10123*C:\\Users\\Bruno\\AppData\\Roaming\\CullinetProgram\\wvxk.zip[@]'
b'128*102*C:\\Users\\Bruno\\AppData\\Roaming\\CullinetProgram\\0E6D47226E5D2DB68FBDD8E5553F3D_FILE